In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
trnN = np.load("/content/drive/MyDrive/Softcom Lab/Chest X-Rays Dataset/train_Normal_128.npy")
trnV = np.load("/content/drive/MyDrive/Softcom Lab/Chest X-Rays Dataset/train_Virus_128.npy")
tstN = np.load("/content/drive/MyDrive/Softcom Lab/Chest X-Rays Dataset/test_Normal_128.npy")
tstV = np.load("/content/drive/MyDrive/Softcom Lab/Chest X-Rays Dataset/test_Virus_128.npy")

In [4]:
print(np.shape(trnN))
print(np.shape(tstN))
print(np.shape(trnV))
print(np.shape(tstV))

(1341, 128, 128, 1)
(234, 128, 128, 1)
(1345, 128, 128, 1)
(148, 128, 128, 1)


In [5]:
trnn = torch.from_numpy(trnN)
tstn = tstN.astype(np.float32)
tstn = torch.from_numpy(tstn)
trnv = torch.from_numpy(trnV)
tstv = torch.from_numpy(tstV)

In [6]:
trnnlbl = torch.zeros(len(trnn))
tstnlbl = torch.zeros(len(tstn))
trnvlbl = torch.ones(len(trnv))
tstvlbl = torch.ones(len(tstv))

In [7]:
#Mixing
trn_data = torch.cat((trnn, trnv), axis=0)
trn_label = torch.cat((trnnlbl, trnvlbl), axis=0)

tst_data = torch.cat((tstn, tstv), axis=0)
tst_label = torch.cat((tstnlbl, tstvlbl), axis=0)

In [8]:
trn_data.shape

torch.Size([2686, 128, 128, 1])

In [9]:
tst_data.shape

torch.Size([382, 128, 128, 1])

In [10]:
trn_data = torch.reshape(trn_data, (2686,1,128,128))

In [11]:
trn_data.shape

torch.Size([2686, 1, 128, 128])

In [12]:
tst_data = torch.reshape(tst_data,(382,1,128,128))

In [13]:
tst_data.shape

torch.Size([382, 1, 128, 128])

In [14]:
torch.manual_seed(20)

In [15]:
x=int(torch.rand(1,)*1000)
print(x)

561


In [16]:
trn_data = [trn_data[i] for i in range(len(trn_data))]
train = torch.stack([d[0] for d in trn_data], dim=0)
#train=train[72:x]        


tst_data = [tst_data[i] for i in range(len(tst_data))]
test = torch.stack([d[0] for d in tst_data], dim=0)       
#test=test[0:382]

train_y = trn_label  
test_y = tst_label 

In [17]:
print(train_y.shape)
print(train.shape)

torch.Size([2686])
torch.Size([2686, 128, 128])


In [18]:
train_loader = torch.utils.data.DataLoader(dataset= (train,train_y) , 
                                           batch_size=100, 
                                           shuffle=True)   # It's better to shuffle the whole training dataset! 

test_loader = torch.utils.data.DataLoader(dataset= (test,test_y), 
                                          batch_size=100, 
                                          shuffle=False)

In [19]:
train_y

tensor([0., 0., 0.,  ..., 1., 1., 1.])

In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=16,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=5, stride=1, padding=2)
        
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.linear_layer_1 = nn.Linear(32*32*32, 512) 
        self.linear_layer_2 = nn.Linear(512, 128)
        self.linear_layer_3 = nn.Linear(128, 2)   
        
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(.25)
        
        # self.flatten = nn.Flatten()
        
    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #print(x.shape)
        
        x = self.cnn_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #print(x.shape)
        
        x = self.flatten(x)
        #print(x.shape)
        
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_3(x)
        #logits = self.sigmoid(x)
        return x

In [21]:
model = NeuralNetwork(16384)
print(model)

NeuralNetwork(
  (cnn_layer_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer_1): Linear(in_features=32768, out_features=512, bias=True)
  (linear_layer_2): Linear(in_features=512, out_features=128, bias=True)
  (linear_layer_3): Linear(in_features=128, out_features=2, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.25, inplace=False)
)


In [22]:
from torchsummary import summary
summary(model,(1,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             416
           Dropout-2         [-1, 16, 128, 128]               0
         LeakyReLU-3         [-1, 16, 128, 128]               0
         MaxPool2d-4           [-1, 16, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]          12,832
           Dropout-6           [-1, 32, 64, 64]               0
         LeakyReLU-7           [-1, 32, 64, 64]               0
         MaxPool2d-8           [-1, 32, 32, 32]               0
           Flatten-9                [-1, 32768]               0
           Linear-10                  [-1, 512]      16,777,728
          Dropout-11                  [-1, 512]               0
        LeakyReLU-12                  [-1, 512]               0
           Linear-13                  [-1, 128]          65,664
          Dropout-14                  [

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [24]:
def trainModel(model, loss_fn, optimizer):
    model.train()

    batch = 100

    loss = 0
    for i in range(train.shape[0]):
      x, y = torch.reshape(train[i],(1,1,128,128)), torch.tensor([train_y[i]], dtype=torch.float)
      label=torch.zeros([1,2,], dtype=torch.float32)
      label[0,int(y.item())]=1
      # Compute prediction error
      pred = model(x)     
      #print(pred)
      #print(label)
      loss += loss_fn(pred, label)
      
      if i>0 and (i+1)%batch == 0:
          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          print(f'Training Loss: {loss.item():.4f}', end="\r")
          loss = 0
    print()

In [25]:
def testModel(model, loss_fn):
    model.eval()

    size = test.shape[0]
    correct=0
    loss = 0
    total =382
    with torch.no_grad():
      for i in range(test.shape[0]):
        x, y = torch.reshape(test[i],(1,1,128,128)), torch.tensor([test_y[i]], dtype=torch.float)
        label=torch.zeros([1,2,], dtype=torch.float32)
        label[0,int(y.item())]=1
        pred = model(x)
        #print(pred)
        predicted = torch.argmax(pred)
        #print(predicted)
        #print(y)


        # Total correct predictions
        correct += (predicted == int(y)).sum()

        loss += loss_fn(pred, label)
      
    loss /= size
    accuracy = 100 * correct.item() / total

    # Print Loss
    print('Loss: {}. Accuracy: {}'.format({loss}, accuracy))

In [26]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainModel(model, loss_fn, optimizer)
    testModel(model, loss_fn)
print("Done!")

Epoch 1
-------------------------------

Loss: {tensor(3.0558e+27)}. Accuracy: 38.7434554973822
Epoch 2
-------------------------------

Loss: {tensor(nan)}. Accuracy: 61.2565445026178
Epoch 3
-------------------------------

Loss: {tensor(nan)}. Accuracy: 61.2565445026178
Epoch 4
-------------------------------

Loss: {tensor(nan)}. Accuracy: 61.2565445026178
Epoch 5
-------------------------------

Loss: {tensor(nan)}. Accuracy: 61.2565445026178
Done!
